In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [8]:
c_list = [[83.16, 122.22, 82.62, 133.50, 81.81, 65.70, 52.89],
          [97.20, 98.28, 75.48, 147.00, 40.50, 70.20, 110.94],
          [123.84, 99.12, 76.16, 130.00, 79.92, 38.40, 101.48],
          [75.60, 123.48, 42.84, 195.00, 72.90, 88.20, 59.34],
          [139.32, 107.10, 78.54, 121.50, 36.45, 65.70, 99.33],
          [83.88, 92.82, 124.10, 134.50, 75.87, 69.90, 48.59]]
c= pd.DataFrame(c_list, index=range(1,7), columns=range(1,8))

c #i: potential sites, j: farms

,1,2,3,4,5,6,7
1,83.16,122.22,82.62,133.5,81.81,65.7,52.89
2,97.20,98.28,75.48,147.0,40.50,70.2,110.94
3,123.84,99.12,76.16,130.0,79.92,38.4,101.48
4,75.60,123.48,42.84,195.0,72.90,88.2,59.34
5,139.32,107.10,78.54,121.5,36.45,65.7,99.33
6,83.88,92.82,124.10,134.5,75.87,69.9,48.59


In [9]:
d= pd.Series([36, 42, 34, 50, 27, 30, 43], index=range(1,8))
d# Demands of farms

1    36
2    42
3    34
4    50
5    27
6    30
7    43
dtype: int64

In [10]:
q= pd.Series([80,90, 110, 120, 100, 120], index= range(1,7))
q # Daily forage throughput of potential sites (supplies)

1     80
2     90
3    110
4    120
5    100
6    120
dtype: int64

In [11]:
f= pd.Series([220, 240, 260, 275, 240, 230], index= range(1,7))
f #daily fixed cost of potential sites                 

1    220
2    240
3    260
4    275
5    240
6    230
dtype: int64

In [12]:
tabu_life = np.round(f/q)
tabu_life

1    3.0
2    3.0
3    2.0
4    2.0
5    2.0
6    2.0
dtype: float64

In [13]:
def simplex(facilities):
    #facilities.sort()
    
    coefs = c.loc[facilities, :].to_numpy().flatten()
    num_i = len(facilities)
    
    smaller_coef = np.zeros((num_i,num_i*7), dtype='float64')
    for i in range(num_i):
        j = 7 * i
        smaller_coef[i][j : j + 7] = d
        
    smaller_rhs = q[facilities]
    
    equal_coef = np.zeros((7, num_i * 7))
    for i in range(7):
        equal_coef[i][i::7] = [1] * num_i
    
    equal_rhs = [1] * 7
    
    res = linprog(coefs,
                  A_ub=smaller_coef, b_ub=smaller_rhs, 
                  A_eq=equal_coef, b_eq=equal_rhs, 
                  method='revised simplex')
    
    x = res.x
    x_split = np.split(x, num_i)
    x_df = pd.DataFrame(x_split, columns=range(1,8), index= facilities)
    
    return res.fun + np.sum(f[facilities]), x_df

In [14]:
def gen_initial_sol():
    demand_satisfied = 0
    total_demand = np.sum(d)
    
    closed_fac = set(range(1,7))
    
    opened_fac = set()
    
    while demand_satisfied < total_demand:
        fac = np.random.choice(list(closed_fac))
        opened_fac.add(fac)
        closed_fac.remove(fac)
        
        demand_satisfied += q[fac]
    
    return simplex(opened_fac)

In [15]:
gen_initial_sol()

(1256.9499999999998,
      1    2    3             4    5    6    7
 1  0.0  0.0  0.0  1.000000e+00  0.0  1.0  0.0
 4  1.0  0.0  1.0  0.000000e+00  1.0  0.0  0.0
 6  0.0  1.0  0.0  1.110223e-16  0.0  0.0  1.0)

In [16]:
def find_neighbours(opened_fac):
    closed_fac = set(range(1,7)).difference(opened_fac)
    
    current_capacity = np.sum(q[opened_fac])
    total_demand = np.sum(d)
    
    neighbours = []
    
    for i in opened_fac:
        for j in closed_fac:
            opened_copy = opened_fac.copy()
            closed_copy = closed_fac.copy()
            if current_capacity - q[i] + q[j] >= total_demand:
                # CLOSE: opened facility i
                opened_copy.remove(i)
                closed_copy.add(i)

                # OPEN: closed facility j
                opened_copy.add(j)
                closed_copy.remove(j)
                
            neighbours.append(opened_copy)
    
    return neighbours

In [17]:
find_neighbours({4,5,6}) #closed 1,2,3

[{1, 5, 6},
 {2, 5, 6},
 {3, 5, 6},
 {1, 4, 6},
 {2, 4, 6},
 {3, 4, 6},
 {1, 4, 5},
 {2, 4, 5},
 {3, 4, 5}]

In [18]:
def tabu_search():
    # Initialization    
    s = gen_initial_sol()
    tabu_list = []
    current_best = s
    
    # Tabu Search
    for t in range(1,101):
        opened_fac = set(s[1].index)
        neighbours = find_neighbours(opened_fac)
        
        # Get the tabu facilities from tabu list
        tabu_fac = {tabu[0] for tabu in tabu_list}
        
        feasible_set = []
        # Add neighbour to feasible set if it does not contain any tabu facility
        for n in neighbours:
            if n.isdisjoint(tabu_fac):
                feasible_set.append(n)
        
        # If feasible set is empty, continue with the next iteration
        if feasible_set == []:
            # Remove tabus which are expired
            for i in tabu_list:
                if i[1] <= t:
                    tabu_list.remove(i)
            continue
            
        # Assign farms to facilities and get the costs
        simplex_results = []
        for sol in feasible_set:
            result = simplex(sol)
            simplex_results.append(result)
        
        new_s = min(simplex_results, key= lambda sol:sol[0])
        
        # Get the indexes (open facilities) of new and old solution, and compare to create a tabu 
        tabu = list(set(s[1].index).difference(set(new_s[1].index)))
        # Appending the expiry time for the tabu
        tabu.append(t + tabu_life[tabu[0]])
        
        # Add tabu to the tabu list
        tabu_list.append(tabu)
        
        # Update best solution if new solution is better
        s = new_s
        if s[0] < current_best[0]:
            current_best = s
        
        # Remove tabus which are expired
        for i in tabu_list:
            if i[1] <= t:
                tabu_list.remove(i)
    return current_best

In [19]:
tabu_search()

(1218.08,
      1    2         3    4    5    6    7
 1  1.0  0.0  0.323529  0.0  0.0  1.0  0.0
 5  0.0  0.0  0.676471  1.0  1.0  0.0  0.0
 6  0.0  1.0  0.000000  0.0  0.0  0.0  1.0)